In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 

In [5]:
from utilities import make_model
class feat_test_mdoel(make_model):
    def __init__(self , name , clf , gamma ,x ,y , train_data_loc):
        self.features = x.columns.to_list() 
        self.train_data_loc = train_data_loc
        make_model.__init__(self , name , clf , gamma , x ,y)

In [ ]:
labels = ['AGN' , 'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
scores = ['precision' , 'recall' , 'f1_score']
models = ['all_feat_v1_0' ,'no_color_0' , 'no_gal_coord_0' , 'no_IR_0' , 'no_mw_0' , 'no_optical_uv_0' ]
for m in models:
    model = joblib.load('models/no_color_0.pkl')

In [2]:
labels = ['AGN' , 'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
scores = ['rep']

In [44]:
import joblib 
model = joblib.load('models/all_feat_v1_0.pkl')
model.result['class_scores'].loc[labels].to_csv('temp.csv')

In [30]:
all_feat = {
    'AGN' : {
        'precision' : [] , 
        'recall' : [],
        'f1' : []
    }
}
for i in range(5):
    model = joblib.load('models/no_optical_uv_0.pkl')
    all_feat['AGN']['precision'].append(model.result['class_scores']['precision_score']['AGN'])
    all_feat['AGN']['recall'].append(model.result['class_scores']['recall_score']['AGN'])
    all_feat['AGN']['f1'].append(model.result['class_scores']['f1_score']['AGN'])
(all_feat['AGN']['precision'])

[0.9587843463780183,
 0.9587843463780183,
 0.9587843463780183,
 0.9587843463780183,
 0.9587843463780183]